In [0]:
pip install kaggle

In [0]:
mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adarshkumarcsb","key":"362b6a937a5c64d80547dc5cf18e845b"}'}

In [0]:
cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

 98% 665M/675M [00:13<00:00, 43.1MB/s]
100% 675M/675M [00:13<00:00, 51.6MB/s]


In [0]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.metrics import FalseNegatives
from keras.metrics import FalsePositives
import os
import zipfile
import gzip
import PIL
from sklearn.model_selection import train_test_split
from PIL import Image
from numpy import asarray
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import tensorflow as tf
from tensorflow.python.framework import ops
tf.compat.v1.disable_eager_execution()
from sklearn import linear_model
import pandas as pd

Using TensorFlow backend.


In [0]:
zip = zipfile.ZipFile("/content/cell-images-for-detecting-malaria.zip","r")
unzipped=zip.extractall()

In [0]:
train_orig_X=[]
train_orig_Y=[]

In [0]:
for filename in os.listdir("/content/cell_images/Parasitized"):
  try:
    image=Image.open("/content/cell_images/Parasitized/"+str(filename))
  except:
    continue
  image=image.resize((100,100))
  arr=np.asarray(image)
  train_orig_X.append(arr)
  train_orig_Y.append(1)
for filename in os.listdir("/content/cell_images/Uninfected"):
  try:
    image=Image.open("/content/cell_images/Uninfected/"+str(filename))
  except:
    continue
  image=image.resize((100,100))
  arr=np.asarray(image)
  train_orig_X.append(arr)
  train_orig_Y.append(0)  

In [0]:
print(len(train_orig_X))
train_orig_X=np.array(train_orig_X)
np.resize(train_orig_X,(27558,100,100,3))
train_orig_X=train_orig_X/255

27558


In [0]:
train_X,test_X,train_Y,test_Y=train_test_split(train_orig_X, train_orig_Y, test_size=0.2, random_state=42)

In [0]:
model=Sequential()
model.add(Conv2D(64,(3,3),strides=(1,1),name="first",activation='relu',input_shape=(100,100,3)))
model.add(BatchNormalization())
model.add(AveragePooling2D((2,2),strides=(2,2)))
model.add(Conv2D(128,(3,3),strides=(1,1),activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D((2,2),strides=(2,2)))
model.add(Conv2D(256,(3,3),strides=(1,1),activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D((2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=256,activation='relu'))
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])
aug_data=ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)
aug_data.fit(train_X)
model.fit(aug_data.flow(train_X,train_Y,batch_size=128),epochs=30,verbose=1)

Epoch 1/30
173/173 [==============================] - 48s 275ms/step - loss: 0.7000 - accuracy: 0.6701
Epoch 2/30
173/173 [==============================] - 44s 252ms/step - loss: 0.4072 - accuracy: 0.8211
Epoch 3/30
173/173 [==============================] - 44s 257ms/step - loss: 0.2634 - accuracy: 0.8989
Epoch 4/30
173/173 [==============================] - 44s 255ms/step - loss: 0.2190 - accuracy: 0.9187
Epoch 5/30
173/173 [==============================] - 44s 254ms/step - loss: 0.1869 - accuracy: 0.9336
Epoch 6/30
173/173 [==============================] - 44s 254ms/step - loss: 0.1781 - accuracy: 0.9370
Epoch 7/30
173/173 [==============================] - 44s 254ms/step - loss: 0.1710 - accuracy: 0.9410
Epoch 8/30
173/173 [==============================] - 44s 253ms/step - loss: 0.1570 - accuracy: 0.9455
Epoch 9/30
173/173 [==============================] - 44s 253ms/step - loss: 0.1480 - accuracy: 0.9488
Epoch 10/30
173/173 [==============================] - 44s 255ms/step - l

In [0]:
model.evaluate(test_X,test_Y)

5512/5512 [==============================] - 3s 455us/step


[0.14632551448016884, 0.9530116319656372]